In [2]:
import json
from pprint import pprint

In [3]:
with open("outputs/test1_output.json","r") as f:
    data = json.load(f)

In [5]:
pprint(data['plan'])

["Enter text 'ssoABRALC' into the input field with id 'okta-signin-username' "
 'on the login page.',
 "Enter text 'QAAdminUser_33a' into the input field with id "
 "'okta-signin-password' on the login page.",
 "Click the submit button with id 'okta-signin-submit' and value 'SIGN IN' to "
 'sign in on the login page.',
 "Click the radio button with id 'radAgents' to select the 'Agents' role on "
 'the Change Role page.',
 "Click the button with id 'btnContinue' and value 'Continue' to proceed on "
 'the Change Role page.',
 "Click the link with id 'header_0_content_0_rItems_hLink_9' and text 'Account "
 "Wizard' in the top navigation after login.",
 "Click the client link with class 'aAccountWizard' and text '1 TBG2 Profile 3 "
 "Over 100K, 1 TBG2 Profile 3 Over 100K' on the Account Wizard page.",
 'Fetch me the Base Class for page functions.',
 'Fetch me the webdriver methods declaration.']


In [ ]:
from code_manager.llm import llm
from code_manager.tools import search_code,project_structure

tool_llm = llm.bind_tools([search_code,project_structure])

In [6]:
query = data['plan'][:6]
recorded_actions = data['recorded_steps']

In [ ]:
from langchain_core.prompts import PromptTemplate

messages = []

prompt = f"""I will give you a **query** and some **recorded actions**.

You have two tools:

1. A **code search tool** – use this to search the codebase using the natural language query.
2. A **codebase structure tool** – use this to explore all files and their organization.

---

### **Your task:**

Use the tools to locate the most likely matching function(s) for the query.
Then, **verify whether the function truly corresponds to the query** using the checklist below.

---

### ✅ **Verification Checklist:**

* [ ] Does the function name or docstring clearly relate to the query?
* [ ] Does the logic inside the function match the behavior described in the query?
* [ ] Is this function called in places related to the recorded actions?
* [ ] Does the function exist in the expected part of the codebase (based on file/module structure)?
* [ ] Are there other functions that might be a better fit?
* [ ] Is there any ambiguity or uncertainty? If so, explain it.

---

### **Input:**

* **Query:** `{query}`
* **Recorded Actions:** `{recorded_actions}`

---

### **Output:**

* The most accurate matching function(s)
* An explanation based on the checklist above
"""

messages.append({"role": "user", "content": prompt})


In [16]:
search_code.invoke(
    {"query_text":"login with okta username and password for ewm","n_results":7}
)

2025-05-20 05:13:49.517 | INFO     | code_manager.tools:search_code:57 - Searching repository at:
2025-05-20 05:13:49.518 | INFO     | code_search.vector_search:_query_embeddings:43 - Response initiated at 1747743229.5185556
2025-05-20 05:13:49.568 | INFO     | code_search.vector_search:_query_embeddings:49 - Response received at 1747743229.568833


------------------------------------------------------------------------------------------------------------------------------------------------------


[{'match_score': 0.762779712677002,
  'lines': [155, 178],
  'node_type': 'method_declaration',
  'text': '\tpublic void loginOWB() {\n\t\tseleniumCore.waitForElementToBeClickable(username, 10, 1);\n\t\tseleniumCore.sendKeys(username, "Madhan.Kumar@assetmark.com");\n\t\tlogger.info("enter username");\n\t\tLoggerService.customLogger("enter username ", "");\n\t\tseleniumCore.click(signIn);\n\t\tseleniumCore.waitForJStoLoad();\n//\t\twaitForLoadingData();\n\t\tseleniumCore.waitForElementToBeClickable(password, 10, 1);\t\t\t\n\t\tseleniumCore.sendKeys(password, "Achivegoal@2025");\n\t\tlogger.info("enter password");\n\t\tLoggerService.customLogger("enter password ", "");\n\t\tseleniumCore.waitForJStoLoad();\n//\t\twaitForLoadingData();\n\t\tseleniumCore.waitForElementToBeClickable(signIn, 10, 1);\n\t\tseleniumCore.click(signIn);\t\t\n\t\tseleniumCore.waitForJStoLoad();\n//\t\twaitForLoadingData();\n\t\tif(seleniumCore.isElementVisible(signIn,10,1)) {\n\t\t\tseleniumCore.click(signIn);\n\t\

In [14]:
pprint(tool_llm.invoke(prompt))

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_O1YWh8dtr6baZdUzX28224kQ', 'function': {'arguments': '{"query_text":"login with okta username and password","n_results":7}', 'name': 'search_code'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 3973, 'total_tokens': 4000, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_3dfb47c1f3', 'id': 'chatcmpl-BZG2MbmCHtSTaPsmMqChoFd82S74a', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'viole

In [8]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt_llm = ChatPromptTemplate.from_messages(
        [
            ("system", "You're a code search assistant. You will be given a query and some recorded actions. Use the tools to find the most relevant function(s) and verify them."),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

In [13]:
chain = prompt_llm | tool_llm
# Run the chain with the query and recorded actions

result = chain.invoke({
    "messages": messages,
})

# Print the result
print("Result:")
pprint(result)

Result:
AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nnPeqO17yLrwKFEZY2KlDGny', 'function': {'arguments': '{"query_text": "get all users", "n_results": 5}', 'name': 'search_code'}, 'type': 'function'}, {'id': 'call_BoayICDZtnTALx3EQZ5qt1Yg', 'function': {'arguments': '{"query_text": "list users", "n_results": 5}', 'name': 'search_code'}, 'type': 'function'}, {'id': 'call_FYEK3OKevxrKGkdeL6pVFnJE', 'function': {'arguments': '{"query_text": "fetch users", "n_results": 5}', 'name': 'search_code'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 200, 'total_tokens': 281, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_3dfb47c1f3', 'id': 'chatcmpl-BZG0H2KVnlDjtrnH6R3zmMfUEK2kG', 'ser